# <font color="#49699E" size=40>Going Viral: Modelling the Epidemic Spread of Simple Contagions</font>
# LEARNING OBJECTIVES
# LEARNING MATERIALS
# INTRODUCTION


# EPIDEMIC SPREAD AND DIFFUSION


# MODELLING SPREADING PROCESSES WITH NDLIB


## Simple Contagions, Beyond Your Neighborhood


## Imports

In [ ]:
import pandas as pd
pd.set_option("display.notebook_repr_html", False)
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.utils import multi_runs

from dcss.plotting import custom_seaborn
from dcss.networks import *

custom_seaborn()

## Simulating a Small World


In [ ]:
population_size = 300
G = nx.watts_strogatz_graph(population_size, 4, 0.15)
G.name = "A Simulated Small World"

print(nx.info(G))

In [ ]:
layout = nx.nx_pydot.graphviz_layout(G)

fig, ax = plt.subplots(figsize=(12, 12))
nx.draw(G,
        pos=layout,
        node_color='gray',
        edge_color='gray',
        node_size=100,
        width=.5)
plt.show()

# SIMPLE CONTAGIONS AND EPIDEMIC SPREAD


## The SIR Model


### Initializing and Configuring the SIR Model


In [ ]:
def sir_model(network, beta, gamma, fraction_infected):
    model = ep.SIRModel(network)

    config = mc.Configuration()
    config.add_model_parameter('beta', beta)
    config.add_model_parameter('gamma', gamma)
    config.add_model_parameter("fraction_infected", fraction_infected)
    
    model.set_initial_status(config)
    return model 

In [ ]:
sir_model_1 = sir_model(G, beta=0.05, gamma=0.01, fraction_infected=0.1)
sir_model_1

### Executing the Simulations


In [ ]:
%%capture 

sir_1_iterations = sir_model_1.iteration_bunch(200, node_status=True)

In [ ]:
[iteration['node_count'] for iteration in sir_1_iterations][:10]

In [ ]:
def simulation_overview(iteration_results, network, prop=True):
    population_size = network.number_of_nodes()

    trends = []
    deltas = []

    for iteration in iteration_results:
        trends.append(iteration['node_count'])
        deltas.append(iteration['status_delta'])

    columns = ['Susceptible', 'Infected', 'Removed']

    # trends DF
    trends = pd.DataFrame(trends)
    trends.columns = columns
    if prop is True:
        trends = trends.div(population_size)

    # deltas DF
    deltas = pd.DataFrame(deltas)
    deltas.columns = columns

    return trends, deltas

In [ ]:
sir_1_trends, sir_1_deltas = simulation_overview(sir_1_iterations, G)    
sir_1_trends.head()

In [ ]:
fig, ax = plt.subplots()
sns.lineplot(data=sir_1_trends)

ax.set(xlabel='Iteration / step in time', ylabel='Proportion of nodes')
sns.despine()
plt.legend()
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,8), sharex=True)
sns.lineplot(ax = ax1, data=sir_1_deltas['Susceptible'], color='gray')
sns.lineplot(ax = ax2, data=sir_1_deltas['Infected'], color='gray')
sns.lineplot(ax = ax3, data=sir_1_deltas['Removed'], color='gray')
    
# emphasize deviations from 0
ax1.axhline(0, color='crimson')
ax2.axhline(0, color='crimson')
ax3.axhline(0, color='crimson')

ax3.set(xlabel='Iteration')
plt.show()  

### Wait, Who Did We Just Infect?


### Let's Do That 500 More Times


In [ ]:
%%capture
trends = multi_runs(sir_model_1, execution_number=500, iteration_number=200, infection_sets=None, nprocesses=4)

In [ ]:
visualize_trends(trends, network=G, proportion=True, return_data=False)

## From Simulated Societies to Denmark


In [ ]:
bluetooth_contact = pd.read_csv("../data/copenhagen_networks_study/cns_bluetooth_filtered.csv", sep=',')
bluetooth_contact.head()

In [ ]:
g_bluetooth_contact = nx.from_pandas_edgelist(bluetooth_contact, 'user_a', 'user_b', create_using=nx.Graph())
g_bluetooth_contact.name = 'CNS Bluetooth Contact'
print(nx.info(g_bluetooth_contact))

In [ ]:
layout = nx.nx_pydot.graphviz_layout(g_bluetooth_contact)

In [ ]:
for_contour = pd.DataFrame([v for k,v in layout.items()])

fig, ax = plt.subplots(figsize=(12, 12))
sns.kdeplot(x=for_contour[0],y=for_contour[1])

nx.draw(g_bluetooth_contact,
        pos=layout,
        node_color='gray',
        edge_color='gray',
        node_size=100,
        width=.5,
       alpha=.3)

plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.ecdfplot(pd.Series(dict(g_bluetooth_contact.degree())))
sns.despine()
ax.set(xlabel='Node degree')
plt.show()

In [ ]:
import random
import copy

random.seed(42)

def rand_infection_set(network, frac):
    node_list = list(network.nodes())
    return random.sample(node_list, int(round(len(node_list)*frac, 0))) # randomly select nodes from node_list without replacement

infect_sets = [rand_infection_set(g_bluetooth_contact, 0.1) for x in range(500)]

In [ ]:
%%capture
sir_model_2 = sir_model(g_bluetooth_contact,
                        beta=0.05,
                        gamma=0.01,
                        fraction_infected=0.1)

sir_model_2_trends = multi_runs(sir_model_2,
                                execution_number=500,
                                iteration_number=300,
                                infection_sets=infect_sets,
                                nprocesses=4)

In [ ]:
visualize_trends(sir_model_2_trends, network=g_bluetooth_contact, proportion=True, return_data=False)

In [ ]:
%%capture
sir_model_3 = sir_model(g_bluetooth_contact,
                        beta=0.01,
                        gamma=0.01,
                        fraction_infected=0.1)

sir_model_3_trends = multi_runs(sir_model_3,
                                execution_number=500,
                                iteration_number=300,
                                infection_sets=infect_sets,
                                nprocesses=4)

In [ ]:
visualize_trends(sir_model_3_trends, network=g_bluetooth_contact, proportion=True, return_data=False)

## Simple Contagions on Dynamic Networks


# CONCLUSION
## Key Points 
